In [1]:
# Load libraries
import pandas as pd
import numpy as np

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load mapping from restauarant ids to yelp ids
id_map = pd.read_csv("Data/restaurant_ids_to_yelp_ids.csv")
id_dict = {}

# Each Yelp ID may correspond to up to 4 Boston IDs
# So create a dictionary with Yelp IDs being keys and Boston IDs being values.
# Much of this code block borrowed from driven data blog.
for i, row in id_map.iterrows():
    # get the Boston ID
    boston_id = row["restaurant_id"]
    
    # get the non-null Yelp IDs
    non_null_mask = ~pd.isnull(row.ix[1:])
    yelp_ids = row[1:][non_null_mask].values
    
    for yelp_id in yelp_ids:
        id_dict[yelp_id] = boston_id

In [3]:
# Read in review data
with open("Data/yelp_boston_academic_dataset_updated/yelp_academic_dataset_review.json", 'r') as review_file:
    # the file is not actually valid json since each line is an individual
    # dict -- we will add brackets on the very beginning and ending in order
    # to make this an array of dicts and join the array entries with commas
    review_json = '[' + ','.join(review_file.readlines()) + ']'

# read in the json as a DataFrame
reviews = pd.read_json(review_json)

# drop columns that we won't use
reviews = reviews.drop(['review_id', 'type'], axis=1)

# replace yelp business_id with boston restaurant_id
map_to_boston_ids = lambda yelp_id: id_dict[yelp_id] if yelp_id in id_dict else np.nan
reviews.business_id = reviews.business_id.map(map_to_boston_ids)
reviews.head()

,business_id,date,stars,text,user_id,votes
0,N6Ok7qOx,2005-08-26 00:00:00,3,This is a pretty typical cafe. The sandwiches...,P_Mk0ygOilLJo4_WEvabAA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
1,N6Ok7qOx,2005-11-23 00:00:00,3,I agree with other reviewers - this is a prett...,TNJRTBrl0yjtpAACr1Bthg,"{u'funny': 0, u'useful': 0, u'cool': 0}"
2,N6Ok7qOx,2005-11-23 00:00:00,3,"Decent enough food, but very overpriced. Just ...",H_mngeK3DmjlOu595zZMsA,"{u'funny': 0, u'useful': 0, u'cool': 0}"
3,N6Ok7qOx,2006-02-24 00:00:00,5,The muffins are great...esp the blueberry! I ...,xjgebo5Q1j9gfu2dAV8vmQ,"{u'funny': 0, u'useful': 0, u'cool': 0}"
4,N6Ok7qOx,2007-09-06 00:00:00,3,"Well, well, well, look at me reviewing the res...",j069r9VPOnXxEXiw8TFnLw,"{u'funny': 3, u'useful': 2, u'cool': 2}"


In [4]:
# Let's unpack the votes column and create columns for funny_votes, useful_votes, and cool_votes.
reviews['funny_votes'] = np.nan
reviews['useful_votes'] = np.nan
reviews['cool_votes'] = np.nan

funny = []
useful = []
cool = []

for x in reviews.votes:
    funny.append(x['funny'])
    useful.append(x['useful'])
    cool.append(x['useful'])

reviews['funny_votes'] = pd.Series(funny)
reviews['useful_votes'] = pd.Series(useful)
reviews['cool_votes'] = pd.Series(cool)
                  
reviews = reviews.drop(['votes'], axis=1)

In [5]:
# Change business_id to restaurant_id
reviews.rename(columns={'business_id':'restaurant_id'}, inplace=True)

# drop restaurants not found in boston data
reviews = reviews[pd.notnull(reviews.restaurant_id)]

# Change datatype of date column to datetime
reviews['date'] = pd.to_datetime(reviews['date'], format='%Y-%m-%d')

print reviews.shape
reviews.head()

(228805, 8)


,restaurant_id,date,stars,text,user_id,funny_votes,useful_votes,cool_votes
0,N6Ok7qOx,2005-08-26 00:00:00,3,This is a pretty typical cafe. The sandwiches...,P_Mk0ygOilLJo4_WEvabAA,0,0,0
1,N6Ok7qOx,2005-11-23 00:00:00,3,I agree with other reviewers - this is a prett...,TNJRTBrl0yjtpAACr1Bthg,0,0,0
2,N6Ok7qOx,2005-11-23 00:00:00,3,"Decent enough food, but very overpriced. Just ...",H_mngeK3DmjlOu595zZMsA,0,0,0
3,N6Ok7qOx,2006-02-24 00:00:00,5,The muffins are great...esp the blueberry! I ...,xjgebo5Q1j9gfu2dAV8vmQ,0,0,0
4,N6Ok7qOx,2007-09-06 00:00:00,3,"Well, well, well, look at me reviewing the res...",j069r9VPOnXxEXiw8TFnLw,3,2,2


In [7]:
# Read in labeled training data
train_labels = pd.read_csv("Data/AllViolations.csv", index_col=0)
train_labels.rename(columns={'date': 'inspect_date'}, inplace = True)
train_labels['inspect_date'] = pd.to_datetime(train_labels['inspect_date'], format='%Y-%m-%d')
train_labels.head()

,inspect_date,restaurant_id,*,**,***
20619,2006-04-04 00:00:00,6VOp6a3L,7,0,2
10144,2006-10-04 00:00:00,1JEbP4oR,0,0,0
19634,2006-10-16 00:00:00,Pk3w02od,6,1,2
24797,2006-10-19 00:00:00,1JEbP4oR,0,0,0
20253,2006-11-17 00:00:00,N6OkmQ3x,0,0,0


In [9]:
# Read in submission format data
submission = pd.read_csv("Data/PhaseIISubmission.csv", index_col=0)
submission.rename(columns={'date': 'inspect_date'}, inplace=True)
submission['inspect_date'] = pd.to_datetime(submission['inspect_date'], format='%Y-%m-%d')
submission.head()

,inspect_date,restaurant_id,*,**,***
id,,,,,
42065,2015-07-07 00:00:00,6VOp6a3L,0,0,0
73703,2015-07-07 00:00:00,1JEbP4oR,0,0,0
19212,2015-07-07 00:00:00,Pk3w02od,0,0,0
15332,2015-07-07 00:00:00,N6OkmQ3x,0,0,0
59579,2015-07-07 00:00:00,v5oJKbOB,0,0,0


In [10]:
# Are any restauarants in the submission file also in the training sample?
submission_restaurants = submission['restaurant_id']
training_restaurants = train_labels[['restaurant_id', '*']]

submission_restaurants = pd.DataFrame(submission_restaurants.unique())
submission_restaurants['one_star'] = pd.Series(0*1687)

submission_restaurants.rename(columns={0:'restaurant_id'}, inplace=True)

overlap = pd.merge(submission_restaurants, training_restaurants, how='left')

In [18]:
# There are no restaurants that do not overlap.
overlap['*'].isnull().any()

False

In [20]:
# Let's create a dataframe of past inspections
train_inspection = train_labels.copy()
train_inspection['One_Star_Lag1'] = [0]*train_inspection.shape[0]
train_inspection['Two_Star_Lag1'] = [0]*train_inspection.shape[0]
train_inspection['Three_Star_Lag1'] = [0]*train_inspection.shape[0]

train_inspection['One_Star_Days_Lag1'] = [0]*train_inspection.shape[0]
train_inspection['Two_Star_Days_Lag1'] = [0]*train_inspection.shape[0]
train_inspection['Three_Star_Days_Lag1'] = [0]*train_inspection.shape[0]

train_inspection['One_Star_Lag2'] = [0]*train_inspection.shape[0]
train_inspection['Two_Star_Lag2'] = [0]*train_inspection.shape[0]
train_inspection['Three_Star_Lag2'] = [0]*train_inspection.shape[0]

train_inspection['One_Star_Days_Lag2'] = [0]*train_inspection.shape[0]
train_inspection['Two_Star_Days_Lag2'] = [0]*train_inspection.shape[0]
train_inspection['Three_Star_Days_Lag2'] = [0]*train_inspection.shape[0]

train_inspection['One_Star_Avg'] = [0]*train_inspection.shape[0]
train_inspection['Two_Star_Avg'] = [0]*train_inspection.shape[0]
train_inspection['Three_Star_Avg'] = [0]*train_inspection.shape[0]

addedColumns = train_inspection.columns[5:]

train_inspection.head()

,inspect_date,restaurant_id,*,**,***,One_Star_Lag1,Two_Star_Lag1,Three_Star_Lag1,One_Star_Days_Lag1,Two_Star_Days_Lag1,Three_Star_Days_Lag1,One_Star_Lag2,Two_Star_Lag2,Three_Star_Lag2,One_Star_Days_Lag2,Two_Star_Days_Lag2,Three_Star_Days_Lag2,One_Star_Avg,Two_Star_Avg,Three_Star_Avg
20619,2006-04-04 00:00:00,6VOp6a3L,7,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10144,2006-10-04 00:00:00,1JEbP4oR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19634,2006-10-16 00:00:00,Pk3w02od,6,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24797,2006-10-19 00:00:00,1JEbP4oR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20253,2006-11-17 00:00:00,N6OkmQ3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# Fill in train_inspection dataframe
for i, (pid,row) in enumerate(train_inspection.iterrows()):
    train_restaurant = train_labels[train_labels['restaurant_id']==row['restaurant_id']]
    train_restaurant = train_restaurant[train_restaurant['inspect_date'] < row['inspect_date']]
    if train_restaurant.shape[0] == 0:
        train_inspection.loc[pid,addedColumns] = [0]*len(addedColumns)
    elif train_restaurant.shape[0] == 1:
        train_restaurant['DaysTilInspection'] = np.abs(row['inspect_date'] - train_restaurant['inspect_date'])
        train_restaurant['DaysTilInspection'] = train_restaurant['DaysTilInspection'].apply(lambda x: x/np.timedelta64(1, 'D')) 
        train_inspection.loc[pid,['One_Star_Lag1', 'Two_Star_Lag1', 'Three_Star_Lag1']] = train_restaurant.iloc[0,2:5].values
        train_inspection.loc[pid,['One_Star_Avg', 'Two_Star_Avg', 'Three_Star_Avg']] = train_restaurant.iloc[0,2:5].values
        train_inspection.loc[pid,'One_Star_Days_Lag1'] = train_restaurant.iloc[0,5]
        train_inspection.loc[pid,'Two_Star_Days_Lag1'] = train_restaurant.iloc[0,5]
        train_inspection.loc[pid,'Three_Star_Days_Lag1'] = train_restaurant.iloc[0,5]
    else: 
        train_restaurant['DaysTilInspection'] = np.abs(row['inspect_date'] - train_restaurant['inspect_date'])
        train_restaurant['DaysTilInspection'] = train_restaurant['DaysTilInspection'].apply(lambda x: x/np.timedelta64(1, 'D'))
        train_restaurant.sort('DaysTilInspection', inplace=True)
        train_inspection.loc[pid,['One_Star_Lag1', 'Two_Star_Lag1', 'Three_Star_Lag1']] = train_restaurant.iloc[0,2:5].values
        train_inspection.loc[pid,'One_Star_Days_Lag1'] = train_restaurant.iloc[0,5]
        train_inspection.loc[pid,'Two_Star_Days_Lag1'] = train_restaurant.iloc[0,5]
        train_inspection.loc[pid,'Three_Star_Days_Lag1'] = train_restaurant.iloc[0,5]
        train_inspection.loc[pid,['One_Star_Lag2', 'Two_Star_Lag2', 'Three_Star_Lag2']] = train_restaurant.iloc[1,2:5].values
        train_inspection.loc[pid,'One_Star_Days_Lag2'] = train_restaurant.iloc[1,5]
        train_inspection.loc[pid,'Two_Star_Days_Lag2'] = train_restaurant.iloc[1,5]
        train_inspection.loc[pid,'Three_Star_Days_Lag2'] = train_restaurant.iloc[1,5]
        train_inspection.loc[pid, 'One_Star_Avg'] = train_restaurant['*'].mean()
        train_inspection.loc[pid, 'Two_Star_Avg'] = train_restaurant['**'].mean()
        train_inspection.loc[pid, 'Three_Star_Avg'] = train_restaurant['***'].mean()
    
    if i % 2500 == 0:
        print '{} out of {}'.format(i, len(train_inspection))

0 out of 34879
2500 out of 34879
5000 out of 34879
7500 out of 34879
10000 out of 34879
12500 out of 34879
15000 out of 34879
17500 out of 34879
20000 out of 34879
22500 out of 34879
25000 out of 34879
27500 out of 34879
30000 out of 34879
32500 out of 34879


In [22]:
# Add rows for new data into train_inspection
train_inspection['One_Star_Reviews'] = [0]*train_inspection.shape[0]
train_inspection['Two_Star_Reviews'] = [0]*train_inspection.shape[0]
train_inspection['Three_Star_Reviews'] = [0]*train_inspection.shape[0]
train_inspection['Four_Star_Reviews'] = [0]*train_inspection.shape[0]
train_inspection['Five_Star_Reviews'] = [0]*train_inspection.shape[0]
train_inspection['Avg_Stars'] = [0]*train_inspection.shape[0]
train_inspection['Num_Reviews'] = [0]*train_inspection.shape[0]

train_inspection['Num_Funny_Votes'] = [0]*train_inspection.shape[0]
train_inspection['Num_Useful_Votes'] = [0]*train_inspection.shape[0]
train_inspection['Num_Cool_Votes'] = [0]*train_inspection.shape[0]

addedReviewCols = train_inspection.columns[20:]

In [23]:
# Let's add Yelp review features.

for i, (pid,row) in enumerate(train_inspection.iterrows()):
    restaurant_reviews = reviews[reviews['restaurant_id']==row['restaurant_id']]
    restaurant_reviews = restaurant_reviews[restaurant_reviews['date'] < row['inspect_date']]
    if restaurant_reviews.shape[0] == 0:
        train_inspection.loc[pid,addedReviewCols] = [0]*len(addedReviewCols)
    train_inspection.loc[pid,'One_Star_Reviews'] = sum(restaurant_reviews['stars']==1)
    train_inspection.loc[pid,'Two_Star_Reviews'] = sum(restaurant_reviews['stars']==2)
    train_inspection.loc[pid,'Three_Star_Reviews'] = sum(restaurant_reviews['stars']==3)
    train_inspection.loc[pid,'Four_Star_Reviews'] = sum(restaurant_reviews['stars']==4)
    train_inspection.loc[pid,'Five_Star_Reviews'] = sum(restaurant_reviews['stars']==5)
    train_inspection.loc[pid,'Avg_Stars'] = restaurant_reviews['stars'].mean()
    train_inspection.loc[pid,'Num_Reviews'] = restaurant_reviews['stars'].count()
    train_inspection.loc[pid,'Num_Funny_Votes'] = restaurant_reviews['funny_votes'].sum()
    train_inspection.loc[pid,'Num_Useful_Votes'] = restaurant_reviews['useful_votes'].sum()
    train_inspection.loc[pid,'Num_Cool_Votes'] = restaurant_reviews['cool_votes'].sum()
    
    if i % 2500 == 0:
        print '{} out of {}'.format(i, len(train_inspection))

0 out of 34879
2500 out of 34879
5000 out of 34879
7500 out of 34879
10000 out of 34879
12500 out of 34879
15000 out of 34879
17500 out of 34879
20000 out of 34879
22500 out of 34879
25000 out of 34879
27500 out of 34879
30000 out of 34879
32500 out of 34879


In [28]:
# Prepare submission data
X_train = train_inspection.iloc[:,5:]
X_train.fillna(0,inplace=True)
X_train = X_train.as_matrix().astype(np.float)
y1 = train_inspection.iloc[:,2].values
y2 = train_inspection.iloc[:,3].values
y3 = train_inspection.iloc[:,4].values

In [30]:
X_train

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    1.,    1.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ..., 
       [  12.,    1.,    2., ...,    8.,   17.,   17.],
       [   0.,    0.,    0., ...,   16.,   31.,   31.],
       [   8.,    1.,    0., ...,   49.,  105.,  105.]])

In [31]:
from sklearn.ensemble import RandomForestRegressor as RF

clf1 = RF()
clf1.fit(X_train, y1)

clf2 = RF()
clf2.fit(X_train, y2)

clf3 = RF()
clf3.fit(X_train, y3)

RandomForestRegressor(bootstrap=True, compute_importances=None,
           criterion='mse', max_depth=None, max_features='auto',
           min_density=None, min_samples_leaf=1, min_samples_split=2,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0)

In [32]:
# Prepare submission data
submission_inspection = submission.copy()
submission_inspection['One_Star_Lag1'] = [0]*submission_inspection.shape[0]
submission_inspection['Two_Star_Lag1'] = [0]*submission_inspection.shape[0]
submission_inspection['Three_Star_Lag1'] = [0]*submission_inspection.shape[0]

submission_inspection['One_Star_Days_Lag1'] = [0]*submission_inspection.shape[0]
submission_inspection['Two_Star_Days_Lag1'] = [0]*submission_inspection.shape[0]
submission_inspection['Three_Star_Days_Lag1'] = [0]*submission_inspection.shape[0]

submission_inspection['One_Star_Lag2'] = [0]*submission_inspection.shape[0]
submission_inspection['Two_Star_Lag2'] = [0]*submission_inspection.shape[0]
submission_inspection['Three_Star_Lag2'] = [0]*submission_inspection.shape[0]

submission_inspection['One_Star_Days_Lag2'] = [0]*submission_inspection.shape[0]
submission_inspection['Two_Star_Days_Lag2'] = [0]*submission_inspection.shape[0]
submission_inspection['Three_Star_Days_Lag2'] = [0]*submission_inspection.shape[0]

submission_inspection['One_Star_Avg'] = [0]*submission_inspection.shape[0]
submission_inspection['Two_Star_Avg'] = [0]*submission_inspection.shape[0]
submission_inspection['Three_Star_Avg'] = [0]*submission_inspection.shape[0]

submission_inspection.head()

,inspect_date,restaurant_id,*,**,***,One_Star_Lag1,Two_Star_Lag1,Three_Star_Lag1,One_Star_Days_Lag1,Two_Star_Days_Lag1,Three_Star_Days_Lag1,One_Star_Lag2,Two_Star_Lag2,Three_Star_Lag2,One_Star_Days_Lag2,Two_Star_Days_Lag2,Three_Star_Days_Lag2,One_Star_Avg,Two_Star_Avg,Three_Star_Avg
id,,,,,,,,,,,,,,,,,,,,
42065,2015-07-07 00:00:00,6VOp6a3L,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
73703,2015-07-07 00:00:00,1JEbP4oR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19212,2015-07-07 00:00:00,Pk3w02od,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15332,2015-07-07 00:00:00,N6OkmQ3x,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59579,2015-07-07 00:00:00,v5oJKbOB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
for i, (pid,row) in enumerate(submission_inspection.iterrows()):
    submission_restaurant = train_labels[train_labels['restaurant_id']==row['restaurant_id']]
    submission_restaurant = submission_restaurant[submission_restaurant['inspect_date'] < row['inspect_date']]
    if submission_restaurant.shape[0] == 0:
        submission_inspection.loc[pid,addedColumns] = [0]*len(addedColumns)
    elif submission_restaurant.shape[0] == 1:
        submission_restaurant['DaysTilInspection'] = np.abs(row['inspect_date'] - submission_restaurant['inspect_date'])
        submission_restaurant['DaysTilInspection'] = submission_restaurant['DaysTilInspection'].apply(lambda x: x/np.timedelta64(1, 'D')) 
        submission_inspection.loc[pid,['One_Star_Lag1', 'Two_Star_Lag1', 'Three_Star_Lag1']] = submission_restaurant.iloc[0,2:5].values
        submission_inspection.loc[pid,['One_Star_Avg', 'Two_Star_Avg', 'Three_Star_Avg']] = submission_restaurant.iloc[0,2:5].values
        submission_inspection.loc[pid,'One_Star_Days_Lag1'] = submission_restaurant.iloc[0,5]
        submission_inspection.loc[pid,'Two_Star_Days_Lag1'] = submission_restaurant.iloc[0,5]
        submission_inspection.loc[pid,'Three_Star_Days_Lag1'] = submission_restaurant.iloc[0,5]
    else: 
        submission_restaurant['DaysTilInspection'] = np.abs(row['inspect_date'] - submission_restaurant['inspect_date'])
        submission_restaurant['DaysTilInspection'] = submission_restaurant['DaysTilInspection'].apply(lambda x: x/np.timedelta64(1, 'D'))
        submission_restaurant.sort('DaysTilInspection', inplace=True)
        submission_inspection.loc[pid,['One_Star_Lag1', 'Two_Star_Lag1', 'Three_Star_Lag1']] = submission_restaurant.iloc[0,2:5].values
        submission_inspection.loc[pid,'One_Star_Days_Lag1'] = submission_restaurant.iloc[0,5]
        submission_inspection.loc[pid,'Two_Star_Days_Lag1'] = submission_restaurant.iloc[0,5]
        submission_inspection.loc[pid,'Three_Star_Days_Lag1'] = submission_restaurant.iloc[0,5]
        submission_inspection.loc[pid,['One_Star_Lag2', 'Two_Star_Lag2', 'Three_Star_Lag2']] = submission_restaurant.iloc[1,2:5].values
        submission_inspection.loc[pid,'One_Star_Days_Lag2'] = submission_restaurant.iloc[1,5]
        submission_inspection.loc[pid,'Two_Star_Days_Lag2'] = submission_restaurant.iloc[1,5]
        submission_inspection.loc[pid,'Three_Star_Days_Lag2'] = submission_restaurant.iloc[1,5]
        submission_inspection.loc[pid, 'One_Star_Avg'] = submission_restaurant['*'].mean()
        submission_inspection.loc[pid, 'Two_Star_Avg'] = submission_restaurant['**'].mean()
        submission_inspection.loc[pid, 'Three_Star_Avg'] = submission_restaurant['***'].mean()
    
    if i % 2500 == 0:
        print '{} out of {}'.format(i, len(submission_inspection))

0 out of 80281
2500 out of 80281
5000 out of 80281
7500 out of 80281
10000 out of 80281
12500 out of 80281
15000 out of 80281
17500 out of 80281
20000 out of 80281
22500 out of 80281
25000 out of 80281
27500 out of 80281
30000 out of 80281
32500 out of 80281
35000 out of 80281
37500 out of 80281
40000 out of 80281
42500 out of 80281
45000 out of 80281
47500 out of 80281
50000 out of 80281
52500 out of 80281
55000 out of 80281
57500 out of 80281
60000 out of 80281
62500 out of 80281
65000 out of 80281
67500 out of 80281
70000 out of 80281
72500 out of 80281
75000 out of 80281
77500 out of 80281
80000 out of 80281


In [34]:
# Add rows for new data into train_inspection
submission_inspection['One_Star_Reviews'] = [0]*submission_inspection.shape[0]
submission_inspection['Two_Star_Reviews'] = [0]*submission_inspection.shape[0]
submission_inspection['Three_Star_Reviews'] = [0]*submission_inspection.shape[0]
submission_inspection['Four_Star_Reviews'] = [0]*submission_inspection.shape[0]
submission_inspection['Five_Star_Reviews'] = [0]*submission_inspection.shape[0]
submission_inspection['Avg_Stars'] = [0]*submission_inspection.shape[0]
submission_inspection['Num_Reviews'] = [0]*submission_inspection.shape[0]

submission_inspection['Num_Funny_Votes'] = [0]*submission_inspection.shape[0]
submission_inspection['Num_Useful_Votes'] = [0]*submission_inspection.shape[0]
submission_inspection['Num_Cool_Votes'] = [0]*submission_inspection.shape[0]

addedReviewCols = submission_inspection.columns[20:]

In [35]:
for i, (pid,row) in enumerate(submission_inspection.iterrows()):
    restaurant_reviews = reviews[reviews['restaurant_id']==row['restaurant_id']]
    restaurant_reviews = restaurant_reviews[restaurant_reviews['date'] < row['inspect_date']]
    if restaurant_reviews.shape[0] == 0:
        submission_inspection.loc[pid,addedReviewCols] = [0]*len(addedReviewCols)
    submission_inspection.loc[pid,'One_Star_Reviews'] = sum(restaurant_reviews['stars']==1)
    submission_inspection.loc[pid,'Two_Star_Reviews'] = sum(restaurant_reviews['stars']==2)
    submission_inspection.loc[pid,'Three_Star_Reviews'] = sum(restaurant_reviews['stars']==3)
    submission_inspection.loc[pid,'Four_Star_Reviews'] = sum(restaurant_reviews['stars']==4)
    submission_inspection.loc[pid,'Five_Star_Reviews'] = sum(restaurant_reviews['stars']==5)
    submission_inspection.loc[pid,'Avg_Stars'] = restaurant_reviews['stars'].mean()
    submission_inspection.loc[pid,'Num_Reviews'] = restaurant_reviews['stars'].count()
    submission_inspection.loc[pid,'Num_Funny_Votes'] = restaurant_reviews['funny_votes'].sum()
    submission_inspection.loc[pid,'Num_Useful_Votes'] = restaurant_reviews['useful_votes'].sum()
    submission_inspection.loc[pid,'Num_Cool_Votes'] = restaurant_reviews['cool_votes'].sum()
    
    if i % 2500 == 0:
        print '{} out of {}'.format(i, len(submission_inspection))

0 out of 80281
2500 out of 80281
5000 out of 80281
7500 out of 80281
10000 out of 80281
12500 out of 80281
15000 out of 80281
17500 out of 80281
20000 out of 80281
22500 out of 80281
25000 out of 80281
27500 out of 80281
30000 out of 80281
32500 out of 80281
35000 out of 80281
37500 out of 80281
40000 out of 80281
42500 out of 80281
45000 out of 80281
47500 out of 80281
50000 out of 80281
52500 out of 80281
55000 out of 80281
57500 out of 80281
60000 out of 80281
62500 out of 80281
65000 out of 80281
67500 out of 80281
70000 out of 80281
72500 out of 80281
75000 out of 80281
77500 out of 80281
80000 out of 80281


In [36]:
submission_inspection.head()

,inspect_date,restaurant_id,*,**,***,One_Star_Lag1,Two_Star_Lag1,Three_Star_Lag1,One_Star_Days_Lag1,Two_Star_Days_Lag1,Three_Star_Days_Lag1,One_Star_Lag2,Two_Star_Lag2,Three_Star_Lag2,One_Star_Days_Lag2,Two_Star_Days_Lag2,Three_Star_Days_Lag2,One_Star_Avg,Two_Star_Avg,Three_Star_Avg,One_Star_Reviews,Two_Star_Reviews,Three_Star_Reviews,Four_Star_Reviews,Five_Star_Reviews,Avg_Stars,Num_Reviews,Num_Funny_Votes,Num_Useful_Votes,Num_Cool_Votes
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
42065,2015-07-07 00:00:00,6VOp6a3L,0,0,0,5,1,2,99,99,99,5,1,2,106,106,106,5.461538,0.461538,1.461538,13,19,23,64,43,3.648148,162,33,83,83
73703,2015-07-07 00:00:00,1JEbP4oR,0,0,0,4,1,0,2342,2342,2342,2,1,0,2357,2357,2357,6.894737,1.105263,1.052632,3,0,5,4,1,3.000000,13,0,3,3
19212,2015-07-07 00:00:00,Pk3w02od,0,0,0,4,0,0,110,110,110,4,0,0,124,124,124,4.696970,0.484848,1.181818,1,0,3,3,6,4.000000,13,2,13,13
15332,2015-07-07 00:00:00,N6OkmQ3x,0,0,0,0,0,0,919,919,919,1,0,2,1806,1806,1806,3.230769,0.000000,0.769231,4,5,7,4,3,2.869565,23,6,15,15
59579,2015-07-07 00:00:00,v5oJKbOB,0,0,0,5,2,4,175,175,175,5,2,4,182,182,182,4.043478,0.782609,1.956522,38,8,19,9,2,2.065789,76,32,47,47


In [37]:
X_submission = submission_inspection.iloc[:,5:]
X_submission.fillna(0,inplace=True)
X_submission = X_submission.as_matrix().astype(np.float)

In [38]:
print X_train.shape
print X_submission.shape

(34879, 25)
(80281, 25)


In [39]:
one_star_pred = clf1.predict(X_submission)
two_star_pred = clf2.predict(X_submission)
three_star_pred = clf3.predict(X_submission)

In [45]:
submission['*'] = one_star_pred.round().astype(int)
submission['**'] = two_star_pred.astype(int)
submission['***'] = three_star_pred.astype(int)

In [46]:
submission = submission.rename(columns={'inspect_date': 'date'})
submission.head()

,date,restaurant_id,*,**,***
id,,,,,
42065,2015-07-07 00:00:00,6VOp6a3L,4,0,1
73703,2015-07-07 00:00:00,1JEbP4oR,0,0,0
19212,2015-07-07 00:00:00,Pk3w02od,4,0,1
15332,2015-07-07 00:00:00,N6OkmQ3x,2,0,0
59579,2015-07-07 00:00:00,v5oJKbOB,5,0,1


In [50]:
submission['**'].value_counts()

0    78093
1     2115
2       73
dtype: int64

In [51]:
submission.to_csv('FinalSubmission2015-07-04-1.csv')